In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DecimalType
from pyspark.sql.functions import col
from decimal import Decimal

# 1. Создайте SparkSession
spark = SparkSession.builder \
    .appName("Lab1_BasicOperations_v1") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# 2. Создайте данные
data = [
    ("Igor", 42, "Engineer", Decimal(200000.00)), 
    ("Olga", 18, "Manager", Decimal(80000.00)),
    ("Kate", 20, "Manager", Decimal(80000.00)),
    ("Maxim", 55, "CIO", Decimal(400000.00)),
    ("Tanya", 30, "Secretar", Decimal(50000.00)),
    ("Leonid", 24, "Manager", Decimal(80000.00))
]

schema = StructType([
    StructField("name", StringType(), False),
    StructField("age", IntegerType(), False),
    StructField("job", StringType(), True),
    StructField("salary", DecimalType(10, 2), True)
])

df = spark.createDataFrame(data, schema)

# 3. Выполните трансформации
df_filtered = df.filter((col("salary") > 100000.00) & (col("age") > 30)).select(
    col("name"), col("salary")
)

# 4. Выполните действия
df_filtered.show()

# 5. Покажите план выполнения
df_filtered.explain()

# 6. Сохраните результат
df_filtered.write.csv("../../data/lab01_v1.txt")

# 7. Остановите SparkSession
spark.stop()

+-----+---------+
| name|   salary|
+-----+---------+
| Igor|200000.00|
|Maxim|400000.00|
+-----+---------+

== Physical Plan ==
*(1) Project [name#8, salary#11]
+- *(1) Filter (isnotnull(salary#11) AND ((cast(salary#11 as double) > 100000.0) AND (age#9 > 30)))
   +- *(1) Scan ExistingRDD[name#8,age#9,job#10,salary#11]


